**“Benchmark Dataset for Mid-Price Forecasting of Limit Order Book Data with Machine Learning Methods,”**

The paper extracted normalized data representations of time series data for five stocks from the NASDAQ Nordic stock market over a period of ten consecutive days. This process resulted in a dataset comprising around four million time series samples in total, providing a comprehensive market-wide history spanning ten trading days.

**Dataset:**
[Explore the Dataset](https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649)

#### Note:
This exploration phase is intended to transition to Apache Spark, utilizing parallel processing and leveraging available functionality for enhanced analysis and forecasting capabilities.


In [ ]:
# Import the necessary libraries

import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset (sample)
df = pd.load_csv("/dataset/BenchmarkDatasets/NoAuction/3.NoAuction_DecPre/NoAuction_DecPre_Training/Train_Dst_NoAuction_DecPre_CF_7.txt")

In [ ]:
# Look at the data 
print(df.shape)
df.head(3)